In [ ]:
import transformers
from datasets import load_dataset

dataset = load_dataset("squad_it")
DEVICE = "cuda:0"

In [ ]:
from get_model import get_model
model_name = "llamantino"

model, tokenizer = get_model(model_name)
#model = model.to(DEVICE)

In [ ]:
from get_prompt import get_prompt

generate_prompt, stop = get_prompt(model_name)

In [ ]:
def build_question(context, question):
    return f"Dato il seguente testo:\n{context}\nRispondi brevemente a questa domanda:\n{question}"

def build_answer(answer):
    return f"Risposta breve: {answer}"

In [ ]:
import random

def get_shots(dataset, n):
    conversation = []
    for i in range(n):
        elem = random.choice(dataset["train"])
        q_shot = build_question(elem["context"], elem["question"])
        conversation.append(dict(
            role="user",
            text=q_shot
        ))
        conversation.append(dict(
            role="ai",
            text=build_answer(elem["answers"]["text"][0])
        ))
    return conversation

In [ ]:
import torch

def forward_model(*, prompt, model, tokenizer):
    if "pad_token" not in tokenizer.special_tokens_map:
        tokenizer.pad_token = tokenizer.eos_token

    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        padding=True,
    ).input_ids
    with torch.no_grad():
        input_ids = input_ids.to(DEVICE)
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=32,
            pad_token_id=tokenizer.eos_token_id,
            prompt_lookup_num_tokens=10
        )

    ret = []
    for i in range(0, len(output_ids)):
        generated_text = tokenizer.decode(
            output_ids[i],
            skip_special_tokens=True
        )
        generated_text = generated_text[len(prompt[i]):]

        if "\n" in generated_text:
            generated_text = generated_text[:generated_text.index("\n")]
        
        ret.append(generated_text.strip())

    return ret

In [ ]:
import time

t0 = time.time()
out = forward_model(
    prompt=["La vita è"],
    model=model,
    tokenizer=tokenizer,
)
print(time.time() - t0)

print(out)

In [ ]:
import uuid
from tqdm import tqdm

batch_size = 1

predicted_answers = []
theoretical_answers = []

ds = dataset["test"].shuffle(seed=42)

bar = tqdm(ds.iter(batch_size=batch_size), total=len(ds))
total = 0
for elem in bar:
    try:
        ids = elem["id"]
        context = elem["context"]
        question = elem["question"]
        answers = elem["answers"]

        model_inputs = []
        for c, q in zip(context, question):
            model_inputs.append(
                generate_prompt(
                    get_shots(dataset, 3) + [
                        dict(
                            role="user",
                            text=build_question(c, q)
                        )
                    ],
                    do_continue=True
                ) + " " + build_answer("").strip()
            )

        model_outputs = forward_model(
            prompt=model_inputs,
            model=model,
            tokenizer=tokenizer,
        )
        #print(f"DOMANDA:{model_inputs[0][520:]}\nRISPOSTA{model_outputs[0]}\nSOLUZIONE{answers[0]['text']}\n")

        for model_output, id in zip(model_outputs, ids):
            predicted_answers.append(dict(
                id=id,
                prediction_text=model_output,
            ))

        for ans, id in zip(answers, ids):
            theoretical_answers.append(dict(
                id=id,
                answers=ans,
            ))
        
    except Exception as e:
        print(e)
        continue






In [ ]:
import os
import json

# Create the directory if it doesn't exist
os.makedirs('./cache', exist_ok=True)
with open(f"./cache/generated-squad-{model_name}.json", "w") as f:
    json.dump(dict(
        predicted_answers=predicted_answers,
        theoretical_answers=theoretical_answers,
    ), f, indent=4)

In [ ]:
import json
with open(f"./cache/generated-squad-{model_name}.json", "r") as f:
    data = json.load(f)

print(len(data["predicted_answers"]))

In [ ]:
import evaluate

predicted_answers = data["predicted_answers"]
theoretical_answers = data["theoretical_answers"]

metric = evaluate.load("squad")
results = metric.compute(predictions=predicted_answers, references=theoretical_answers)

import time


print("=== REPORT ===")
print("current date:", time.strftime("%d/%m/%Y %H:%M:%S"))
print("Dataset: SQuAD-it")
print("Model:", model_name)
print(results)
print("==========================")